In [1]:
import pandas as pd

import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

from keras import models
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.utils import to_categorical

In [2]:
data = pd.read_csv('icml_face_data.csv')
data.head()

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [4]:
def prepare_data(data):

    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label

In [5]:
train_image_array, train_image_label = prepare_data(data[data[' Usage']=='Training'])
val_image_array, val_image_label = prepare_data(data[data[' Usage']=='PrivateTest'])
test_image_array, test_image_label = prepare_data(data[data[' Usage']=='PublicTest'])

In [6]:
train_images = train_image_array.reshape((train_image_array.shape[0], 48, 48, 1))
train_images = train_images.astype('float32')/255
val_images = val_image_array.reshape((val_image_array.shape[0], 48, 48, 1))
val_images = val_images.astype('float32')/255
test_images = test_image_array.reshape((test_image_array.shape[0], 48, 48, 1))
test_images = test_images.astype('float32')/255

In [11]:
train_labels = to_categorical(train_image_label)
val_labels = to_categorical(val_image_label)
test_labels = to_categorical(test_image_label)

In [12]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [13]:
model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

In [15]:
history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    
                    epochs=12,
                    batch_size=64)

Epoch 1/12
449/449 [==============================] - 61s 135ms/step - loss: 1.6879 - accuracy: 0.3269 - val_loss: 1.5511 - val_accuracy: 0.4087
Epoch 2/12
449/449 [==============================] - 57s 126ms/step - loss: 1.4822 - accuracy: 0.4314 - val_loss: 1.4125 - val_accuracy: 0.4625
Epoch 3/12
449/449 [==============================] - 56s 124ms/step - loss: 1.3507 - accuracy: 0.4862 - val_loss: 1.3291 - val_accuracy: 0.4898
Epoch 4/12
449/449 [==============================] - 56s 124ms/step - loss: 1.2709 - accuracy: 0.5197 - val_loss: 1.2643 - val_accuracy: 0.5104
Epoch 5/12
449/449 [==============================] - 58s 128ms/step - loss: 1.2045 - accuracy: 0.5467 - val_loss: 1.2705 - val_accuracy: 0.5085
Epoch 6/12
449/449 [==============================] - 54s 120ms/step - loss: 1.1529 - accuracy: 0.5663 - val_loss: 1.2490 - val_accuracy: 0.5216
Epoch 7/12
449/449 [==============================] - 53s 117ms/step - loss: 1.1072 - accuracy: 0.5841 - val_loss: 1.2168 - val_ac

In [16]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('test caccuracy:', test_acc)

113/113 [==============================] - 3s 26ms/step - loss: 1.2956 - accuracy: 0.5386
test caccuracy: 0.538590133190155
